# Final Project Template 

## 1) Get your data
You may use any data set(s) you like, so long as they meet these criteria:

* Your data must be publically available for free.
* Your data should be interesting to _you_. You want your final project to be something you're proud of.
* Your data should be "big enough":
    - It should have at least 1,000 rows.
    - It should have enough of columns to be interesting.
    - If you have questions, contact a member of the instructional team.

## 2) Provide a link to your data
Your data is required to be free and open to anyone.
As such, you should have a URL which anyone can use to download your data:

In [474]:
# https://www.ncbi.nlm.nih.gov/pubmed/?term=the+encyclopedia+of+DNA+elements

# 3) Import your data

In the space below, import your data. If your data span multiple files, read them all in. If applicable, merge or append them as needed.

In [477]:
import pandas as pd
import numpy as np 
import csv

from Bio import Entrez
Entrez.email = "eileen.cahill@nih.gov"
handle = Entrez.esearch(db="pubmed", retmax=100, term="genomics", idtype="acc")
record = Entrez.read(handle)
handle.close()

In [480]:
print(type(record))
print(type(handle))
print(len(listofrecords))
print(record)
#print(record['IdList'])


<class 'Bio.Entrez.Parser.DictionaryElement'>
<class '_io.TextIOWrapper'>
1
{'Count': '246378', 'RetMax': '100', 'RetStart': '0', 'IdList': ['32359160', '32359158', '32359157', '32359152', '32359146', '32359007', '32358926', '32358872', '32358784', '32358770', '32358743', '32358605', '32358589', '32358529', '32358503', '32357955', '32357898', '32357861', '32357859', '32357836', '32357832', '32357518', '32357425', '32357373', '32357172', '32357153', '32356861', '32356383', '32356375', '32356309', '32356189', '32355762', '32355734', '32355611', '32355594', '32355559', '32355537', '32355288', '32355211', '32355162', '32355159', '32354994', '32354986', '32354985', '32354967', '32354947', '32354896', '32354366', '32354351', '32354324', '32354297', '32354124', '32354024', '32353943', '32353872', '32353747', '32353626', '32353625', '32353525', '32353479', '32353478', '32353477', '32353476', '32353475', '32353474', '32353450', '32353447', '32353178', '32353123', '32353112', '32353023', '323530

In [482]:
list = []
listrecords = []

for key, value in record.items():
    list = [key, value]
    listrecords.append(list)
print(listrecords)


[['Count', '246378'], ['RetMax', '100'], ['RetStart', '0'], ['IdList', ['32359160', '32359158', '32359157', '32359152', '32359146', '32359007', '32358926', '32358872', '32358784', '32358770', '32358743', '32358605', '32358589', '32358529', '32358503', '32357955', '32357898', '32357861', '32357859', '32357836', '32357832', '32357518', '32357425', '32357373', '32357172', '32357153', '32356861', '32356383', '32356375', '32356309', '32356189', '32355762', '32355734', '32355611', '32355594', '32355559', '32355537', '32355288', '32355211', '32355162', '32355159', '32354994', '32354986', '32354985', '32354967', '32354947', '32354896', '32354366', '32354351', '32354324', '32354297', '32354124', '32354024', '32353943', '32353872', '32353747', '32353626', '32353625', '32353525', '32353479', '32353478', '32353477', '32353476', '32353475', '32353474', '32353450', '32353447', '32353178', '32353123', '32353112', '32353023', '32353016', '32352857', '32352843', '32352579', '32352380', '32352041', '323

In [ ]:
id_list_acquire = listrecords[3]
#print(id_list_acquire)
id_list = []

for id in id_list_acquire[1]:
    id_list.append(int(id))
    #print(id)
print(id_list)

In [486]:
from pubmed_lookup import PubMedLookup as PML
from pubmed_lookup import Publication

email = 'eileen.cahill@nih.gov'

for url in id_list:
    lookup = PML(url, email)
    publication = Publication(lookup)
    print(publication.abstract)


Biodegradation of the phenylurea herbicide linuron appears a specialization within a specific clade of the Variovorax genus. The linuron catabolic ability is likely acquired by horizontal gene transfer but the mechanisms involved are not known. The full genome sequences of six linuron degrading Variovorax strains isolated from geographically distant locations were analyzed to acquire insight into the mechanisms of genetic adaptation towards linuron metabolism. Whole genome sequence analysis confirmed the phylogenetic position of the linuron degraders in a separate clade within Variovorax and indicated that they unlikely originate from a common ancestral linuron degrader. The linuron degraders differentiated from Variovorax strains that do not degrade linuron by the presence of multiple plasmids of 20 to 839 kb, including plasmids of unknown plasmid groups. The linuron catabolic gene clusters showed (i) high conservation and synteny and (ii) strain-dependent distribution among the diffe

TypeError: descriptor 'get' requires a 'dict' object but received a 'str'

## 4) Show me the head of your data.

In [488]:
#df = pd.DataFrame(record)
#df.head()

## 5) Show me the shape of your data

In [489]:
#df.shape()

## 6) Show me the proportion of missing observations for each column of your data

## 7) Give me a problem statement.
Below, write a problem statement. Keep in mind that your task is to tease out relationships in your data and eventually build a predictive model. Your problem statement can be vague, but you should have a goal in mind. Your problem statement should be between one sentence and one paragraph.

In [490]:
def preprocess_text(text)
    text = text.lower()
    processed_tokens = []
    for token in word_tokenize(text)
        if token not in punctuation and token not in stopword_list:
          processed_tokens.append(token)
        return processed_tokens  

SyntaxError: invalid syntax (<ipython-input-490-7306c8c9af96>, line 1)

## 8) What is your _y_-variable?
For final project, you will need to perform a statistical model. This means you will have to accurately predict some y-variable for some combination of x-variables. From your problem statement in part 7, what is that y-variable?

In [491]:
#VS code version